# Vector Embedding

## Vectors

A vector is a mathematical object that has both magnitude and direction. It is often represented as an arrow in a coordinate system, where the length of the arrow represents the magnitude and the direction of the arrow represents the direction.

$$
\vec{v} = \begin{pmatrix} 3 \\ 4 \end{pmatrix}
$$

We will use [Numpy](https://numpy.org/) for fast vector operations.

<img src="img/numpy.png" alt="Numpy" width="400">

In [ ]:
from my_functions import draw_vectors
import numpy as np

v = np.array([3, 4])

draw_vectors([[3, 4, "b"]], [0, 5], [0, 5], "v")

### Vector Norm (Magnitude|Length)

The norm of a vector is a measure of the length of the vector. It is calculated as the square root of the sum of the squares of the vector elements.

$$
\| \vec{v} \| = \sqrt{3^2 + 4^2}=5

In [ ]:
np.linalg.norm(v)

### Vector Normalization

Normalization is the process of scaling individual samples to have unit norm. In case of vectors, it is the process of scaling a vector to have a length of 1. It is done by dividing the vector by its norm. The normalized vector is also called a unit vector written as $\vec{v_0}$.

$$
\vec{v_0} = \frac{\vec{v}}{\| \vec{v} \|} = \frac{\begin{pmatrix} 3 \\ 4 \end{pmatrix}}{5} = \begin{pmatrix} 0.6 \\ 0.8 \end{pmatrix}
$$

$$
\| \vec{v_0} \| = \sqrt{0.6^2 + 0.8^2} = 1
$$

The direction of the vector keeps the same after normalization.

In [ ]:
v0 = v / np.linalg.norm(v)
v0

In [ ]:
draw_vectors([[3, 4, "b"], [0.6, 0.8, "r"]], [0, 5], [0, 5], "v")

### Vector Multiplication (Dot Product)

The dot product of two vectors is a scalar quantity that is calculated as the sum of the products of the corresponding elements of the two vectors. It is also called the inner product. The dot product has a result of zero if the two vectors are orthogonal (perpendicular) to each other.

$$
\vec{v} \cdot \vec{w} = \begin{pmatrix} 3 \\ 4 \end{pmatrix} \cdot \begin{pmatrix} -4 \\ 3 \end{pmatrix} = 3 \cdot (-4) + 4 \cdot 3 = 0
$$

In [ ]:
w = np.array([-4, 3])
a = np.array([2, 1])
print(np.dot(v, w))  # static method
print(v.dot(a))  # instance method

In [ ]:
draw_vectors([[3, 4, "b"], [-4, 3, "r"]], [-4, 3], [0, 6], "orthogonal vectors")

### Angle Between Vectors

The angle is independent of the magnitude of the vectors and is calculated as the inverse cosine of the dot product of the vectors divided by the product of the magnitudes of the vectors.

![Angle Between Vectors](img/angle.png)

$$
\cos(\phi) = \frac{\vec{v} \cdot \vec{w}}{\| \vec{v} \| \cdot \| \vec{w} \|} = \text{cosine similarity}
$$

So if the magnitudes of the vectors are 1, the cosine similarity is the same as the dot product because the division by 1 does not change the value!

$$
\cos(\phi) = \vec{v_0} \cdot \vec{w_0} = \text{cosine similarity}
$$

#### Cosine Similarity

The cosine similarity has a range of [-1,1].

| Cosine Similarity | Angle |
| --- | --- |
| 1 | 0° |
| 0 | 90° |
| -1 | 180° |



### More Dimensions

*Above you have seen nicely drawn 2D vectors. It is possible to draw 3D vectors as well, but it is not possible to draw 4D vectors and higher. But the math works the same way. The dot product and the angle between vectors are calculated the same way.*

## Embeddings

An embedding model takes data like texts, images, or audio and converts it into a vector space. The vector space is a multi-dimensional space where each dimension represents a feature of the data. The embedding model learns to map the data into the vector space in such a way that similar data points are close to each other in the vector space.

![Embedding](img/embedding.png)

Source: https://jkfran.com/introduction-vector-embedding-databases.md

Embeddings are used in many machine learning tasks, such as natural language processing, computer vision, and recommendation systems. Now it's easier to find similar items, compare items, and make predictions based on the data.

### Text Embeddings using [OpenAI's text-embedding-3-large model](https://openai.com/pricing)

The text-embedding-3-large model is a transformer-based model that takes text as input and outputs a vector representation of the text. The model is trained on a large corpus of text data and learns to map the text into a vector space in such a way that similar texts are close to each other in the vector space.

Each vector has **3072 dimensions**.

In [ ]:
import numpy as np
from openai import OpenAI  # OpenAI's Python client
import pandas as pd
from dotenv import load_dotenv
import os

# Create OpenAI client
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


# Define a helper function to calculate embeddings
def get_embedding_vec(input):
    """Returns the embeddings vector for a given input"""
    return (
        client.embeddings.create(
            input=input,
            model="text-embedding-3-large",  # We use the new embeddings model here (announced end of Jan 2024)
            # dimensions=500  You can limit the number of output dimensions with the new embeddings models
        )
        .data[0]
        .embedding
    )

### Basic example

In [ ]:
words = ["dog", "cat", "animal", "house", "castle", "ship", "boat"]
embeddings = [get_embedding_vec(word) for word in words]

**Information about an embedding**

In [ ]:
# Embedding of word "dog"

print("Embedding:", embeddings[0])
print("Dimensions:", len(embeddings[0]))
print("Norm:", np.linalg.norm(embeddings[0]))

As you can see in the example below, the embedding is a list of 3072 float numbers. The model gives us vectors with a norm of nearly 1. The vectors are already normalized and the dot product can be used to calculate the cosine similarity.

Now lets compare "dog" and "boat" with the other words.

In [ ]:
comparisons = {"word": [], "Cosine similarity": []}

for i in range(0, len(words)):
    word = words[i]
    embedding = embeddings[i]
    similarity = np.dot(embeddings[0], embedding)  # Cosine similarity
    comparisons["word"].append(word)
    comparisons["Cosine similarity"].append(similarity)

print("Dog")
pd.DataFrame(comparisons).sort_values("Cosine similarity", ascending=False)

In [ ]:
comparisons = {"word": [], "Cosine similarity": []}

for i in range(0, len(words)):
    word = words[i]
    embedding = embeddings[i]
    similarity = np.dot(embeddings[len(words) - 1], embedding)  # Cosine similarity
    comparisons["word"].append(word)
    comparisons["Cosine similarity"].append(similarity)

print("Boat")
pd.DataFrame(comparisons).sort_values("Cosine similarity", ascending=False)

### More Examples

Here the strength of the embedding model becomes clear.

In [ ]:
def compare(s):
    sim1 = np.dot(get_embedding_vec(s[0]), get_embedding_vec(s[1]))
    sim2 = np.dot(get_embedding_vec(s[0]), get_embedding_vec(s[2]))
    return (sim1, sim2)


sentences = [
    # Although sentences 1 and 2 use different words (soccer, football),
    # the cosine similarity of 1 and 2 is higher compared to 1 and 3 because
    # the meaning of 1 and 2 is more similar.
    [
        "I enjoy playing soccer on weekends.",
        "Football is my favorite sport. Playing it on weekends with friends helps me to relax.",
        "In Austria, people often watch soccer on TV on weekends.",
    ],
    # Here we test whether the OpenAI embedding model "understands", that the
    # contextual meaning of "Java" is different in sentences 1 and 2. Therefore,
    # the cosine similarity of 1 and 3 is higher as both are programming-related.
    [
        "He is interested in Java programming.",
        "He visited Java last summer.",
        "He recently started learning Python programming.",
    ],
    # The next example deals with negation handling. All three sentences are
    # about whether someone likes going to the gym. Sentences 1 and 3 are positive
    # (i.e. like training in the gym), while 2 is not. Therefore, 1 and 3 have
    # a higher cosine similarity.
    [
        "I like going to the gym.",
        "I don't like going to the gym.",
        "I don't dislike going to the gym.",
    ],
    # Let's take a look at ideomatic expressions. Sentences 1 and 2 have very
    # similar meaning. 3 also contains "cats and dogs", but the meaning is different.
    # As a result, cosine similarity between 1 and 2 is higher.
    [
        "It's raining cats and dogs.",
        "The weather is very bad, it's pouring outside.",
        "Cats and dogs don't go outside when it rains.",
    ],
    # The last example demonstrates the limits of embeddings. Berry Harris is
    # a well-known teacher in Jazz. Using "the 6th on the 5th" is typical for him.
    # One must know Berry Harris and the musical theory that he has tought to
    # understand the similarity of the sentences 1 and 2. OpenAI embeddings
    # do not understand that.
    [
        "I like how Barry Harris described Jazz theory.",
        "Playing the 6th on the 5th is an important concept that you must understand.",
        "My friends Barry and Harris often visit me to play computer games.",
    ],
]
print(f"Semantic similarity: {compare(sentences[0])}")
print(f"Contextual meaning: {compare(sentences[1])}")
print(f"Negation handling: {compare(sentences[2])}")
print(f"Idiomatic expressions: {compare(sentences[3])}")
print(f"Knowledge: {compare(sentences[4])}")

### My trip advisor

I will use the text-embedding-3-large model to create a trip advisor. The user can input a text and the model will find the most similar destinations in the database. The database contains 9 different locations with a description. The model will return the most similar locations to the user input.

In [ ]:
descriptions = [
    "Maldives, Maldives: A tropical paradise with pristine beaches. Activities: Snorkeling, Scuba diving, Relaxing on the beach.",
    "Paris, France: The City of Love, famous for its iconic landmarks and art. Activities: Eiffel Tower visit, Louvre Museum, Seine River cruise.",
    "Maui, Hawaii: A beautiful island with stunning beaches and lush landscapes. Activities: Surfing, Hiking, Whale watching.",
    "Cancun, Mexico: A popular beach destination with crystal-clear waters. Activities: Swimming, Snorkeling, Exploring Mayan ruins.",
    "Tokyo, Japan: A vibrant metropolis with a mix of modern and traditional attractions. Activities: Shibuya crossing, Tokyo Disneyland, Tsukiji Fish Market.",
    "Banff National Park, Canada: A breathtaking national park with stunning mountain scenery. Activities: Hiking, Wildlife spotting, Canoeing.",
    "Norwegian Fjords, Norway: A picturesque region with majestic fjords and waterfalls. Activities: Fjord cruises, Hiking, Kayaking.",
    "New York City, USA: The Big Apple, known for its skyscrapers, museums, and diverse culture. Activities: Times Square, Central Park, Statue of Liberty visit.",
    "Plitvice Lakes National Park, Croatia: A natural wonderland with cascading lakes and waterfalls. Activities: Walking trails, Boat rides, Wildlife watching.",
]

destinations = []

for description in descriptions:
    parts = description.split(": ")
    destinations.append(
        {
            "Name": parts[0],
            "Description": parts[1],
            "Embedding": get_embedding_vec(parts[1]),
        }
    )

In [ ]:
search_values = [
    "My next holidays should be in a sunny place with a nice beach",
    "I want to see beautiful nature",
    "In the next summer I would like to visit a famous city with a lot of culture",
]

number = 0  # 0, 1, 2

search_embedding = get_embedding_vec(search_values[number])

data = {"Name": [], "Similarity": []}
for destination in destinations:
    similarity = np.dot(search_embedding, destination["Embedding"])
    data["Name"].append(destination["Name"])
    data["Similarity"].append(similarity)
print(f"Search: {search_values[number]}")
pd.DataFrame(data).sort_values("Similarity", ascending=False)

## Vector Search and the RAG Pattern

Embeddings play a crucial role in _Retrieval-Augmented Generation_ (_RAG_) solutions, an approach in artificial intelligence that combines the capabilities of information retrieval and text generation. In RAG systems, embeddings are used to retrieve relevant information from large datasets or knowledge bases. It is not necessary for these databases to have been included in the original training of the embeddings models. They can be internal data sets that are not publicly accessible on the internet.

In RAG solutions, queries or input texts are transformed into embeddings. Then, the cosine similarity to the document embeddings existing in the database is calculated to identify the most relevant text sections from the database. These retrieved pieces of information are then used by a text generation model like _ChatGPT_ to generate contextually relevant answers or content.

_Vector databases_ play a central role in the functioning of RAG systems. They are designed to efficiently store, index, and query high-dimensional vectors. In the context of RAG solutions and similar systems, vector databases serve as a storage for the embeddings of documents or data pieces that come from a large pool of information. When a user makes a query, this query is first transformed into an embedding vector. The vector database is then used to quickly find the vectors that correspond most closely to this query vector - that is, those documents or pieces of information that show the highest similarity. This process of quickly finding similar vectors in large datasets is known as _Nearest Neighbor Search_.

In [ ]:
search_values = [
    "My next holidays should be in a sunny place with a nice beach",
    "I want to see beautiful nature",
    "In the next summer I would like to visit a famous city with a lot of culture",
    "What is the name of Bart Simpsons sister?",
]

number = 3  # 0, 1, 2

search_embedding = get_embedding_vec(search_values[number])

data = {"Destination": [], "Similarity": []}
for destination in destinations:
    similarity = np.dot(search_embedding, destination["Embedding"])
    data["Destination"].append(destination["Name"] + ": " + destination["Description"])
    data["Similarity"].append(similarity)
print(f"Search: {search_values[number]}")
sorted_result = list(
    pd.DataFrame(data).sort_values("Similarity", ascending=False).head(3)["Destination"]
)
sorted_result

In [ ]:
from string import Template

t = Template(
    """
You are a helpful assistant in a travel agency. Customers are describing
what they want to do in their vacation. Make suggestions based on the
city descriptions provided below. ONLY use the provided city descriptions.
Do NOT use other information sources.

If you cannot generate a meaningful answer based on the given city description,
say "Sorry, I cannot help". If the user's input is not related to finding
a travel location, say "Sorry, I can only help with vacation locations".

===========
$options
===========
"""
)

system_prompt = t.substitute(options="\n\n".join([item for item in sorted_result[:3]]))
print(system_prompt)

In [ ]:
print("ChatGPT-4 Answer:")
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": search_values[number],
        },
    ],
    model="gpt-4-1106-preview",
)
print(chat_completion.choices[0].message.content)

## Local Open Source Embedding models

- [sentence-transformers](https://www.sbert.net)
- [transformers](https://huggingface.co/transformers/)

It is possible to use the sentence-transformers library to create embeddings for text data and images. The library provides pre-trained models that can be used to create embeddings for text data and images and is based on PyTorch. The library also provides a simple API for creating embeddings and performing similarity searches. The models give you the vector as a numpy array so the cosine similarity can be calculated easily.

Just install the python library with pip:

```bash
pip install sentence-transformers
```

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-mpnet-base-v2")

dog = model.encode("dog")

print("Embedding:", dog)
print("Dimensions:", len(dog))
print("Norm:", np.linalg.norm(dog))

In [ ]:
cat = model.encode("cat")

words = ["dog", "cat", "animal", "house", "castle", "ship", "boat"]
print(dog.dot(cat))  # Cosine similarity with numpy dot function

# use sentence-transformers to calculate cosine similarity

embeddings = model.encode(words)

util.dot_score(dog, embeddings).tolist()

Let's try our examples from above with the sentence-transformers library.

In [ ]:
def compare(s):
    sim1 = np.dot(model.encode(s[0]), model.encode(s[1]))
    sim2 = np.dot(model.encode(s[0]), model.encode(s[2]))
    return (sim1, sim2)


sentences = [
    [
        "I enjoy playing soccer on weekends.",
        "Football is my favorite sport. Playing it on weekends with friends helps me to relax.",
        "In Austria, people often watch soccer on TV on weekends.",
    ],
    [
        "He is interested in Java programming.",
        "He visited Java last summer.",
        "He recently started learning Python programming.",
    ],
    [
        "I like going to the gym.",
        "I don't like going to the gym.",
        "I don't dislike going to the gym.",
    ],
    [
        "It's raining cats and dogs.",
        "The weather is very bad, it's pouring outside.",
        "Cats and dogs don't go outside when it rains.",
    ],
    [
        "I like how Barry Harris described Jazz theory.",
        "Playing the 6th on the 5th is an important concept that you must understand.",
        "My friends Barry and Harris often visit me to play computer games.",
    ],
]
print(f"Semantic similarity: {compare(sentences[0])}")
print(f"Contextual meaning: {compare(sentences[1])}")
print(f"Negation handling: {compare(sentences[2])}")
print(f"Idiomatic expressions: {compare(sentences[3])}")
print(f"Knowledge: {compare(sentences[4])}")

## Vector Databases

![Vector Database](img/qdrant.png)

Vector databases are databases that store vectors and allow you to perform similarity searches on the vectors. They are used to store and search for embeddings. The Qdrant vector database is an open-source vector database written in Rust.

Vector database improve the performance of similarity searches by using algorithms that are optimized for searching in high-dimensional vector spaces.

Here you can see how to use the Qdrant vector database to store and search for embeddings.

**Start docker container:**

```bash
docker container run --name qdrant -p 6333:6333 qdrant/qdrant
```

### Connect to the database and create collection

In [ ]:
from qdrant_client import QdrantClient  # Qdrant's Python client

qdrant = QdrantClient("localhost", port=6333)  # Connect to Qdrant

from qdrant_client.http.models import Distance, VectorParams

qdrant.create_collection(
    collection_name="destinations",
    vectors_config=VectorParams(
        size=3072, distance=Distance.DOT
    ),  # Could be Distance.COSINE
)

In [ ]:
from qdrant_client.http.models import PointStruct

points = []

# add points to my array

for i, destination in enumerate(destinations):
    points.append(
        PointStruct(
            id=i,
            vector=destination["Embedding"],
            payload={"destination": destination["Name"]},
        )
    )

# add points to the collection

operation_info = qdrant.upsert(
    collection_name="destinations",
    wait=True,
    points=points,
)

print(operation_info)

In [ ]:
search_result = qdrant.search(
    collection_name="destinations",
    query_vector=get_embedding_vec("I want to see beautiful nature"),
    limit=3,
)

search_result